In [ ]:
    'lstm_units': [256,512],  # Focused values for LSTM units
    'learning_rate': [0.001,0.0001],  # Narrow range for learning rate
    'output_dim': [128,256]  # Output dimensions of Embedding layer

In [ ]:
256 128 0.001

In [ ]:
# Wrap the model with KerasClassifier
model = KerasClassifier(model=create_model, lstm_units=128, learning_rate=0.001, output_dim=128 ,epochs=1, batch_size=32, verbose=1)

In [ ]:
def create_model(lstm_units=128, learning_rate=0.001, dropout_rate=0.3, regularization=0.001, input_length=max_length, input_dim=vocab_size, output_dim=128):
    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length),
        LSTM(units=lstm_units, kernel_regularizer=l2(regularization),input_shape=(input_length, 128)),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    optimizer = Nadam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model  

In [ ]:
    'lstm_units': [128, 256],  # Focused values for LSTM units
    'learning_rate': [0.001, 0.0001],  # Narrow range for learning rate
    'output_dim': [128, 256]  # Output dimensions of Embedding layer

In [ ]:
param_grid = {
    'lstm_units': [128, 256],  # Focused values for LSTM units
    'learning_rate': [0.001, 0.0001],  # Narrow range for learning rate
    'num_layers': [2, 3]  # Number of stacked LSTM layers
}

+ change epoch !!

In [ ]:
    'neurons': [128, 256],  # Starting neurons for the first layer
    'num_layers': [1, 2],  # Varying numbers of layers
    'learning_rate': [0.001, 0.0001],
    'regularization': [0, 0.001]  # Adding regularization to the grid search

In [ ]:
    'lstm_units': [128, 256],  # Focused values for LSTM units
    'learning_rate': [0.001, 0.0001],  # Narrow range for learning rate
    'dropout_rate': [0.3, 0.5],  # Focused range for dropout rate
    'num_layers': [2, 3]  # Number of stacked LSTM layers

In [ ]:
    'lstm_units': [128, 256],  # Focused values for LSTM units
    'learning_rate': [0.001, 0.0001],  # Narrow range for learning rate
    'dropout_rate': [0.3, 0.5],  # Focused range for dropout rate
    'num_layers': [2, 3]  # Number of stacked LSTM layers

In [ ]:
# Create the final model using the best parameters
model_1cTI = create_model(
    neurons=best_params['neurons'],
    num_layers=best_params['num_layers'],
    learning_rate=best_params['learning_rate'],
    regularization=best_params['regularization']
)

In [ ]:
param_grid = {
    'lstm_units': [128, 256],  # Focused values for LSTM units
    'learning_rate': [0.001, 0.0001],  # Narrow range for learning rate
    'dropout_rate': [0.3, 0.5],  # Focused range for dropout rate
    'regularization': [0, 0.001],  # Focused regularization strengths
}

In [ ]:
# Define the model building function
def create_model(neurons=64, dropout_rate=0.5, activation='relu'):
    model = Sequential([
        Dense(neurons, activation=activation, input_shape=(cX_train_vec.shape[1],)),
        Dropout(dropout_rate),
        Dense(int(neurons / 2), activation=activation),  # Automatically reduce neurons for the second layer
        Dense(1, activation='sigmoid')
    ])
    return model

# Wrap the model with KerasClassifier
model = KerasClassifier(build_fn=create_model, neurons=128, dropout_rate=0.1, activation='relu', optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

# Define the grid search parameters
param_grid = {
    'neurons': [128, 64],  # Top layer neurons, second layer will be half
    'dropout_rate': [0.1, 0.3, 0.5],
    'activation': ['sigmoid', 'relu', 'tanh'],
    'optimizer': ['rmsprop', 'adam'],
    'batch_size': [10, 30, 50],
}

# Create the GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3, verbose=1)

# Execute the grid search
grid_result = grid.fit(cX_train_vec, cy_train)

# Print best results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Extract the best parameters
best_params = grid_result.best_params_

# Create the final model using the best parameters
model_1c = create_model(
    layers=best_params['build_fn__layers'],
    neurons_per_layer=best_params['build_fn__neurons_per_layer'],
    dropout_rate=best_params['build_fn__dropout_rate'],
    activation=best_params['build_fn__activation']
)

# Compile the model
model_1c.compile(
    optimizer=best_params['optimizer'],
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Fit the model
model_1c_history = model_1c.fit(
    cX_train_vec, cy_train,
    epochs=300,
    batch_size=best_params['batch_size'],
    validation_split=0.2,  # Using 20% of the training data as validation data
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

# Use the function with your model's history
plot_loss_curves(model_1c_history)

In [ ]:
# Load the Universal Sentence Encoder from TensorFlow Hub
USE_embedding = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", input_shape=[], dtype=tf.string, trainable=False)

# Precompute the embeddings for your training data
rX_train_embeddings = USE_embedding(rX_train)
rX_test_embeddings = USE_embedding(rX_test)

############### Model raw 79.29% #####################

# Enhanced model architecture with additional layers and Batch Normalization
model_4r = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(rX_train_embeddings.shape[1],)),
    tf.keras.layers.Dense(1024, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Using an optimizer with adaptive learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)

# Compile the model
model_4r.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callback for early stopping
callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Number of epochs
epochs = 150

# Train the model with updated configurations
model_4r_history = model_4r.fit(
    rX_train_embeddings, 
    ry_train, 
    batch_size=64, 
    epochs=epochs, 
    validation_split=0.2, 
    callbacks=[callback_es]
)

# Create a function to vizuualize the training and validation loss and accuracy
def plot_loss_curves(history):
    """
    Returns separate loss curves for training and validation metrics.
    Args:
    history: TensorFlow History object.
    """
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    epochs = range(len(history.history['loss']))

    # Plot loss
    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.title('Loss Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='Training Accuracy')
    plt.plot(epochs, val_accuracy, label='Validation Accuracy')
    plt.title('Accuracy Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()


# Use the function with your model's history
plot_loss_curves(model_4r_history)


# Predicting the ratings and measuring the time it takes
start_time = timeit.default_timer()
ry_pred_prob = model_4r.predict(rX_test_embeddings)  # This returns probabilities
end_time = timeit.default_timer()

# Convert probabilities to binary predictions
ry_pred = (ry_pred_prob > 0.5).astype(int)  # Converts probs to 0 or 1 based on threshold 0.5

# Calculate time per instance
m4r_time_per_instance = (end_time - start_time) / len(rX_test_embeddings)

# Calculate accuracy, precision, recall, and f1 score
m4r_accuracy = accuracy_score(ry_test, ry_pred)
m4r_precision = precision_score(ry_test, ry_pred, average='weighted')  # Use 'weighted' to account for label imbalance
m4r_recall = recall_score(ry_test, ry_pred, average='weighted')
m4r_f1 = f1_score(ry_test, ry_pred, average='weighted')

# Display the results in a formatted way
results = pd.DataFrame({
    'Model Performance': ['accuracy', 'precision', 'recall', 'f1_score', 'time/instance'],
    'Results': [f"{m4r_accuracy:.2%}", f"{m4r_precision:.3f}", f"{m4r_recall:.3f}", f"{m4r_f1:.3f}", f"{m4r_time_per_instance:.6f} seconds"]
})

print(results.to_string(index=False))
xs

In [ ]:
######## model raw 78.56% ########

# Enhanced model architecture with additional layers and Batch Normalization
model_4r = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(rX_train_embeddings.shape[1],)),
    tf.keras.layers.Dense(1024, activation='swish',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(512, activation='swish',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(256, activation='swish',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),    
    tf.keras.layers.Dense(128, activation='swish',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Using an optimizer with adaptive learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.000005)

# Compile the model
model_4r.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callback for early stopping
callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Number of epochs
epochs = 150

# Train the model with updated configurations
model_4r_history = model_4r.fit(
    rX_train_embeddings, 
    ry_train, 
    batch_size=32, 
    epochs=epochs, 
    validation_split=0.2, 
    callbacks=[callback_es]

In [ ]:
############### Model raw 79.29% #####################

# Enhanced model architecture with additional layers and Batch Normalization
model_4r = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(rX_train_embeddings.shape[1],)),
    tf.keras.layers.Dense(1024, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Using an optimizer with adaptive learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)

# Compile the model
model_4r.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callback for early stopping
callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Number of epochs
epochs = 150

# Train the model with updated configurations
model_4r_history = model_4r.fit(
    rX_train_embeddings, 
    ry_train, 
    batch_size=32, 
    epochs=epochs, 
    validation_split=0.2, 
    callbacks=[callback_es]
)

In [ ]:
############### Model raw 78.20% #####################
# Enhanced model architecture with additional layers and Batch Normalization
model_4r = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(rX_train_embeddings.shape[1],)),
    tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Using an optimizer with adaptive learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)

# Compile the model
model_4r.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callback for early stopping
callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Number of epochs
epochs = 150

# Train the model with updated configurations
model_4r_history = model_4r.fit(
    rX_train_embeddings, 
    ry_train, 
    batch_size=32, 
    epochs=epochs, 
    validation_split=0.2, 
    callbacks=[callback_es]
)

In [ ]:
########model raw 79.13%%########


# Model architecture with added complexity and refined regularization
model_4r = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(rX_train_embeddings.shape[1],)),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),  # Increased complexity and adjusted regularizer
    tf.keras.layers.Dropout(0.3),  # Slightly reduced dropout
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Optimizer with a lower initial learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)

# Compile the model
model_4r.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting with a slightly more patient approach
callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Number of epochs
epochs = 150

# Train the model
model_4r_history = model_4r.fit(
    rX_train_embeddings, 
    ry_train, 
    batch_size=32, 
    epochs=epochs, 
    validation_split=0.2, 
    callbacks=[callback_es]
)

In [ ]:
########model raw 77.87%########

# Redefine the model without the USE embedding layer
model_4r = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(rX_train_embeddings.shape[1],)),  # Specify input shape based on embedding output
    tf.keras.layers.Dense(256, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model_4r.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Number of epochs
epochs = 150

# Train the model with a callback for early stopping
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model using the precomputed embeddings
model_4r_history = model_4r.fit(rX_train_embeddings, ry_train, batch_size=32, epochs=epochs, validation_split=0.2, callbacks=[callback])


In [ ]:
### for raw , model 1 ####

model_4r = tf.keras.Sequential([
    USE_embedding,
    tf.keras.layers.Dense(96, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)),  # Adjusted number of neurons and regularization
    tf.keras.layers.Dropout(0.5),  # Adjusted dropout rate
    tf.keras.layers.Dense(48, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)),  # Adjusted number of neurons and regularization
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)


model_4r.compile(optimizer=optimizer,
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

# Number of epochs
epochs = 15


# Train the model with a callback for early stopping
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history_model_4r = model_4r.fit(rX_train, ry_train, batch_size=32, epochs=epochs, validation_split=0.2, callbacks=[callback])


In [ ]:
######### 4.  #########  50% ######### 4.

model_4c = tf.keras.Sequential([
    USE_embedding,
    tf.keras.layers.Dense(64, activation='swish', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01)),  # Reduced neurons, added L1 & L2 regularization
    tf.keras.layers.Dropout(0.6),  # Increased dropout
    tf.keras.layers.Dense(32, activation='swish', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01)),  # Reduced neurons, added L1 & L2 regularization
    tf.keras.layers.Dropout(0.6),  # Increased dropout
    tf.keras.layers.Dense(1, activation='sigmoid')


lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0005,
    decay_steps=10000,
    decay_rate=0.9)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model_4c.compile(optimizer=optimizer,
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

# Number of epochs
epochs = 15


# Train the model with a callback for early stopping
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history_model_4c = model_4c.fit(cX_train, cy_train, batch_size=32, epochs=epochs, validation_split=0.2, callbacks=[callback])

In [ ]:
######### 3.  #########  75.12% ######### 3. 

model_4c = tf.keras.Sequential([
    USE_embedding,
    tf.keras.layers.Dense(128, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.01)),  # Reduced neurons and added L2 regularization
    tf.keras.layers.Dropout(0.5),  # Increased dropout
    tf.keras.layers.Dense(64, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.01)),  # Further reduced neurons
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_4c.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
                 loss='binary_crossentropy',
                 metrics=['accuracy'])


In [ ]:
######### 2.  #########  74.27% ######### 2.  #########

model_4c = tf.keras.Sequential([
    USE_embedding,  # Universal Sentence Encoder
    tf.keras.layers.Dense(256, activation='swish'),  # Increased neurons, change activation function
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),  # Increased dropout
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer
])

model_4c.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),  # Reduced learning rate
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

In [ ]:
######### 1.  #########


model_4c = tf.keras.Sequential([
    USE_embedding,  # Add the USE embedding layer
    tf.keras.layers.Dense(128, activation='relu'),  # Add a Dense layer
    tf.keras.layers.Dropout(0.3),  # Add Dropout
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer
])

model_4c.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define the batch size and number of epochs
batch_size = 32
epochs = 1

# Train the model
history = model_4c.fit(cX_train, cy_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)  # Using 20% of the data for validation

In [ ]:
store_results('Model_4c', m4c_accuracy, m4c_precision, m4c_recall, m4c_f1, m4c_time_per_instance)

# View the consolidated results
print(results_df)

In [ ]:
# Generate the plot
plot_model(model_4c, show_shapes=True, show_layer_names=True)

In [ ]:
model_5=keras.Sequential([
    USE_embedding,
    layers.Dense(64,activation='relu'),
    layers.Dense(1,activation='sigmoid',dtype=tf.float32)
])

In [ ]:
history = final_model.fit(
    cX_train_vec,
    cy_train,
    epochs=10,  # You might want to choose an optimal number of epochs based on prior experiments
    batch_size=32,
    validation_split=0.2  # Optional if you want to monitor validation metrics
)

In [ ]:
import gensim
from gensim.models import Word2Vec
import numpy as np

# Assuming 'cleaned_train' and 'cleaned_test' are pre-processed and tokenized
# Tokenize reviews if not already done
cleaned_train['tokenized_reviews'] = cleaned_train['review'].apply(gensim.utils.simple_preprocess)
cleaned_test['tokenized_reviews'] = cleaned_test['review'].apply(gensim.utils.simple_preprocess)

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=cleaned_train['tokenized_reviews'], vector_size=100, window=5, min_count=2, workers=4)

# Function to vectorize texts
def vectorize_reviews(reviews, model):
    vectorized = []
    for tokens in reviews:
        # Filter tokens to ensure the model has a vector for them
        valid_tokens = [token for token in tokens if token in model.wv]
        if valid_tokens:
            # If there are valid tokens, calculate the mean of their vectors
            review_vec = np.mean([model.wv[token] for token in valid_tokens], axis=0)
        else:
            # If no valid tokens, use a zero vector
            review_vec = np.zeros(model.vector_size)
        vectorized.append(review_vec)
    return np.array(vectorized)


# Vectorize training and testing data
cX_train_vec = vectorize_reviews(cleaned_train['tokenized_reviews'], word2vec_model)
cX_test_vec = vectorize_reviews(cleaned_test['tokenized_reviews'], word2vec_model)


In [2]:
import matplotlib.pyplot as plt

# Assuming 'results_df' is your DataFrame containing the results
data = results_df.values
columns = results_df.columns

# Create a figure and a subplot
fig, ax = plt.subplots(figsize=(10, 0.6 + 0.3 * len(data)))  # Smaller height factor to reduce vertical space
ax.axis('off')  # Hide the axes

# Table
table = ax.table(
    cellText=data,
    colLabels=columns,
    cellLoc='center',
    loc='center',
    colColours=["#56B4E9"]*len(columns),  # Color for the column headers
    colWidths=[0.16, 0.12, 0.12, 0.12, 0.12, 0.36]  # Adjust column widths to fit your content
)
table.auto_set_font_size(False)
table.set_fontsize(10)  # Adjust font size if necessary
table.scale(1, 1.4)  # Scale table size, reduce x-scaling to minimize horizontal space

# Adjust layout to make room for the table:
plt.subplots_adjust(left=0.1, right=0.9, top=1, bottom=0.05)  # Minimize margins as much as possible

plt.show()

NameError: name 'results_df' is not defined

In [ ]:
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None
    
def handle_negation(text):
    if text is None:
        return text
    # Pattern to detect negations followed by an alphanumeric word
    negation_pattern = re.compile(r'\b(not|no|never|none|nothing|nowhere|neither|nobody|cannot)\b[\s]*(\w+)')
    text = negation_pattern.sub(lambda x: f'{x.group(1)}_{x.group(2)}', text)
    return text

def full_clean_text(text):
    # If text is None or empty, return it as it is
    if text is None or text == '':
        return text
    # Handle negation
    text = handle_negation(text)
    # Remove emojis
    text = emoji.replace_emoji(text, replace='')
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    words = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # POS tagging
    nltk_pos_tagged = nltk.pos_tag(words)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_pos_tagged)
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word, tag) if tag else word for word, tag in wordnet_tagged]
    return ' '.join(lemmatized_words)

def raw_clean_text(text):
    # If text is None or empty, return it as it is
    if text is None or text == '':
        return text
    # Tokenize text
    words = word_tokenize(text)
    return ' '.join(words)



In [ ]:
from sklearn.model_selection import train_test_split

# Split the original dataset into training and test sets
train_rws, test_rws = train_test_split(rws, test_size=0.2, random_state=42)

# Apply the thorough cleaning to the training and test sets
# Create new DataFrame for cleaned versions including ratings
cleaned_train = pd.DataFrame({
    'review': train_rws['review'].apply(full_clean_text),
    'rating': train_rws['rating']
})
cleaned_test = pd.DataFrame({
    'review': test_rws['review'].apply(full_clean_text),
    'rating': test_rws['rating']
})

# Apply the minimal cleaning to the training and test sets
# Create new DataFrame for raw versions including ratings
raw_train = pd.DataFrame({
    'review': train_rws['review'].apply(raw_clean_text),
    'rating': train_rws['rating']
})
raw_test = pd.DataFrame({
    'review': test_rws['review'].apply(raw_clean_text),
    'rating': test_rws['rating']
})

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import re

# Ensure that NLTK's resources are downloaded
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Separate positive and negative reviews
positive_reviews = rws[rws['rating'] == 1]['review']
negative_reviews = rws[rws['rating'] == 0]['review']

# Apply the cleaning and tokenization function
positive_tokens = positive_reviews.apply(raw_clean_text).sum()
negative_tokens = negative_reviews.apply(raw_clean_text).sum()

# Count the frequencies of words
positive_freq = Counter(positive_tokens)
negative_freq = Counter(negative_tokens)

# Get the 10 most common words in positive and negative reviews
most_common_positive = positive_freq.most_common(10)
most_common_negative = negative_freq.most_common(10)

# Separate the words and their counts for the plot
positive_words, positive_counts = zip(*most_common_positive)
negative_words, negative_counts = zip(*most_common_negative)

# Create subplots
fig, ax = plt.subplots(1, 2, figsize=(18, 6))

# Plot for positive reviews
ax[0].barh(positive_words, positive_counts, color='green')
ax[0].set_title('Top words in Positive Reviews')
ax[0].invert_yaxis()  # Labels read top-to-bottom

# Plot for negative reviews
ax[1].barh(negative_words, negative_counts, color='red')
ax[1].set_title('Top words in Negative Reviews')
ax[1].invert_yaxis()  # Labels read top-to-bottom

# Display the plot
plt.tight_layout()
plt.show()
